In [62]:
import os
import zipfile
from zipfile import ZipFile
from os import path
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, to_timestamp, unix_timestamp,mean, to_date, month
from pyspark.sql import types
import numpy as np

In [3]:
def read_zip_files_into_dataframes(zip_file_paths):
    df_list = []
    for file_path in zip_file_paths:
        print(f"Reading for {file_path}")
        zip_file = ZipFile(file_path)
        for text_file in zip_file.infolist():
            if text_file.filename.endswith(
                ".csv"
            ) and not text_file.filename.startswith("__MACOSX"):
                df = pd.read_csv(zip_file.open(text_file.filename))
                # add a column for the year based on the filename
                year = int(path.basename(file_path).split(".")[0].split("_")[-2])
                df['year'] = year
                df_list.append(df)
    return df_list

In [4]:
def get_file_paths(root_directory, folder_name, file_extension):
    file_paths = []
    file_directory = os.path.join(root_directory, folder_name)
    for root, _, files in os.walk(file_directory):
        for file_name in files:
            if file_extension in file_name:
                file_paths.append(os.path.join(root, file_name))
    return file_paths

In [5]:
def write_to_csv(df, file_path):
  # set mode to overwrite
  df.write.mode("overwrite").csv(file_path, header=True)

In [6]:
current_directory = os.getcwd()

In [7]:
current_directory

'c:\\Users\\fahmi\\projects\\data-engineering-practice\\Exercises\\Exercise-6'

In [8]:
zip_file_paths = get_file_paths(current_directory, r"data", "zip")

In [9]:
zip_file_paths

['c:\\Users\\fahmi\\projects\\data-engineering-practice\\Exercises\\Exercise-6\\data\\Divvy_Trips_2019_Q4.zip',
 'c:\\Users\\fahmi\\projects\\data-engineering-practice\\Exercises\\Exercise-6\\data\\Divvy_Trips_2020_Q1.zip']

In [10]:
pd.DataFrame.iteritems = pd.DataFrame.items

In [11]:
pdf = read_zip_files_into_dataframes(zip_file_paths)

Reading for c:\Users\fahmi\projects\data-engineering-practice\Exercises\Exercise-6\data\Divvy_Trips_2019_Q4.zip
Reading for c:\Users\fahmi\projects\data-engineering-practice\Exercises\Exercise-6\data\Divvy_Trips_2020_Q1.zip


In [12]:
pdf[0]

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,2215,940.0,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,Subscriber,Male,1987.0,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,6328,258.0,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,Subscriber,Male,1998.0,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,3003,850.0,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Female,1991.0,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,3275,"2,350.0",313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,Subscriber,Male,1990.0,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,5294,"1,867.0",210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,Subscriber,Male,1987.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
704049,25962900,2019-12-31 23:56:13,2020-01-01 00:15:45,2196,"1,172.0",112,Green St & Randolph St,225,Halsted St & Dickens Ave,Subscriber,Male,1981.0,2019
704050,25962901,2019-12-31 23:56:34,2020-01-01 00:22:08,4877,"1,533.0",90,Millennium Park,90,Millennium Park,Subscriber,Male,1992.0,2019
704051,25962902,2019-12-31 23:57:05,2020-01-01 00:05:46,863,520.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Male,1967.0,2019
704052,25962903,2019-12-31 23:57:11,2020-01-01 00:05:45,2637,514.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Female,1970.0,2019


In [21]:
# drop columns that are not neede
df_2019 = pdf[0]

In [14]:
df_2019

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,2215,940.0,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,Subscriber,Male,1987.0,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,6328,258.0,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,Subscriber,Male,1998.0,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,3003,850.0,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Female,1991.0,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,3275,"2,350.0",313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,Subscriber,Male,1990.0,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,5294,"1,867.0",210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,Subscriber,Male,1987.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
704049,25962900,2019-12-31 23:56:13,2020-01-01 00:15:45,2196,"1,172.0",112,Green St & Randolph St,225,Halsted St & Dickens Ave,Subscriber,Male,1981.0,2019
704050,25962901,2019-12-31 23:56:34,2020-01-01 00:22:08,4877,"1,533.0",90,Millennium Park,90,Millennium Park,Subscriber,Male,1992.0,2019
704051,25962902,2019-12-31 23:57:05,2020-01-01 00:05:46,863,520.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Male,1967.0,2019
704052,25962903,2019-12-31 23:57:11,2020-01-01 00:05:45,2637,514.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Female,1970.0,2019


In [22]:
df_2019[['birthyear']] = df_2019[['birthyear']].astype('Int64')

In [23]:
df_2019.head()

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,2215,940.0,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,Subscriber,Male,1987,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,6328,258.0,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,Subscriber,Male,1998,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,3003,850.0,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Female,1991,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,3275,"2,350.0",313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,Subscriber,Male,1990,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,5294,"1,867.0",210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,Subscriber,Male,1987,2019


In [24]:
df_2019[["trip_id"]] = df_2019[["trip_id"]].astype(str)

In [25]:
df_2019.dtypes

trip_id              object
start_time           object
end_time             object
bikeid                int64
tripduration         object
from_station_id       int64
from_station_name    object
to_station_id         int64
to_station_name      object
usertype             object
gender               object
birthyear             Int64
year                  int64
dtype: object

In [26]:
df_2019['start_time'] = pd.to_datetime(df_2019['start_time'])

In [29]:
df_2019['end_time'] = pd.to_datetime(df_2019['end_time'])

In [30]:
df_2019=df_2019.rename(columns={'trip_id' : "ride_id", "start_time": "started_at", "end_time": "ended_at", "from_station_id": "start_station_id", "from_station_name": "start_station_name", "to_station_id": "end_station_id", "to_station_name": "end_station_name"})

In [31]:
df_2019

,ride_id,started_at,ended_at,bikeid,tripduration,start_station_id,start_station_name,end_station_id,end_station_name,usertype,gender,birthyear,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,2215,940.0,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,Subscriber,Male,1987,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,6328,258.0,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,Subscriber,Male,1998,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,3003,850.0,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,Subscriber,Female,1991,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,3275,"2,350.0",313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,Subscriber,Male,1990,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,5294,"1,867.0",210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,Subscriber,Male,1987,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
704049,25962900,2019-12-31 23:56:13,2020-01-01 00:15:45,2196,"1,172.0",112,Green St & Randolph St,225,Halsted St & Dickens Ave,Subscriber,Male,1981,2019
704050,25962901,2019-12-31 23:56:34,2020-01-01 00:22:08,4877,"1,533.0",90,Millennium Park,90,Millennium Park,Subscriber,Male,1992,2019
704051,25962902,2019-12-31 23:57:05,2020-01-01 00:05:46,863,520.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Male,1967,2019
704052,25962903,2019-12-31 23:57:11,2020-01-01 00:05:45,2637,514.0,623,Michigan Ave & 8th St,52,Michigan Ave & Lake St,Subscriber,Female,1970,2019


In [35]:
df_2020 = pdf[1]

In [38]:
df_2020

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,year
0,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326.0,41.9665,-87.6884,41.9671,-87.6674,member,2020
1,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318.0,41.9616,-87.6660,41.9542,-87.6644,member,2020
2,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117.0,41.9401,-87.6455,41.9402,-87.6530,member,2020
3,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24.0,41.8846,-87.6319,41.8918,-87.6206,member,2020
4,943BC3CBECCFD662,docked_bike,2020-01-30 08:37:16,2020-01-30 08:42:48,Clinton St & Lake St,66,Wells St & Hubbard St,212.0,41.8856,-87.6418,41.8899,-87.6343,member,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426882,ADDAA33CEBCAE733,docked_bike,2020-03-10 10:40:06,2020-03-10 10:40:07,HQ QR,675,HQ QR,675.0,41.8899,-87.6803,41.8899,-87.6803,casual,2020
426883,82B10FA3994BC66A,docked_bike,2020-03-07 15:25:55,2020-03-07 16:14:03,Rush St & Superior St,161,Sheridan Rd & Irving Park Rd,240.0,41.8958,-87.6259,41.9542,-87.6544,member,2020
426884,AA0D5AAA0B59C8AA,docked_bike,2020-03-01 13:12:38,2020-03-01 13:38:29,Clark St & Lincoln Ave,141,Ashland Ave & Division St,210.0,41.9157,-87.6346,41.9035,-87.6677,casual,2020
426885,3296360A7BC20FB8,docked_bike,2020-03-07 18:02:45,2020-03-07 18:13:18,Franklin St & Illinois St,672,Stetson Ave & South Water St,264.0,41.8910,-87.6355,41.8868,-87.6223,member,2020


In [45]:
overlapping_columns = df_2020.columns.intersection(df_2019.columns).tolist()

In [47]:
df_2019_subset = df_2019.loc[:, overlapping_columns]

In [48]:
df_2020_subset = df_2020.loc[:, overlapping_columns]

In [50]:
subset_list = [df_2019_subset, df_2020_subset]

In [51]:
combined_df = pd.concat(subset_list, ignore_index=True)

In [52]:
combined_df.head()

,ride_id,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,Sheffield Ave & Kingsbury St,20,Leavitt St & Armitage Ave,309.0,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,Throop (Loomis) St & Taylor St,19,Morgan St & Polk St,241.0,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,Milwaukee Ave & Grand Ave,84,Wabash Ave & Grand Ave,199.0,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,Lakeview Ave & Fullerton Pkwy,313,Kedzie Ave & Palmer Ct,290.0,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,Ashland Ave & Division St,210,Western Ave & Congress Pkwy,382.0,2019


In [ ]:
combined_df[['end_station_id']] = combined_df[['end_station_id']].astype('Int64')
combined_df[['start_station_id']] = combined_df[['start_station_id']].astype('Int64')

In [54]:
combined_df

,ride_id,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,Sheffield Ave & Kingsbury St,20,Leavitt St & Armitage Ave,309,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,Throop (Loomis) St & Taylor St,19,Morgan St & Polk St,241,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,Milwaukee Ave & Grand Ave,84,Wabash Ave & Grand Ave,199,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,Lakeview Ave & Fullerton Pkwy,313,Kedzie Ave & Palmer Ct,290,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,Ashland Ave & Division St,210,Western Ave & Congress Pkwy,382,2019
...,...,...,...,...,...,...,...,...
1130936,ADDAA33CEBCAE733,2020-03-10 10:40:06,2020-03-10 10:40:07,HQ QR,675,HQ QR,675,2020
1130937,82B10FA3994BC66A,2020-03-07 15:25:55,2020-03-07 16:14:03,Rush St & Superior St,161,Sheridan Rd & Irving Park Rd,240,2020
1130938,AA0D5AAA0B59C8AA,2020-03-01 13:12:38,2020-03-01 13:38:29,Clark St & Lincoln Ave,141,Ashland Ave & Division St,210,2020
1130939,3296360A7BC20FB8,2020-03-07 18:02:45,2020-03-07 18:13:18,Franklin St & Illinois St,672,Stetson Ave & South Water St,264,2020


In [55]:
cols = combined_df.columns.tolist()

In [56]:
cols 

['ride_id',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'year']

In [89]:
combined_df = combined_df[['ride_id','started_at', 'ended_at','start_station_id','start_station_name','end_station_id','end_station_name','year']]

In [59]:
combined_df

,ride_id,started_at,ended_at,start_station_id,start_station_name,end_station_id,end_station_name,year
0,25223640,2019-10-01 00:01:39,2019-10-01 00:17:20,20,Sheffield Ave & Kingsbury St,309,Leavitt St & Armitage Ave,2019
1,25223641,2019-10-01 00:02:16,2019-10-01 00:06:34,19,Throop (Loomis) St & Taylor St,241,Morgan St & Polk St,2019
2,25223642,2019-10-01 00:04:32,2019-10-01 00:18:43,84,Milwaukee Ave & Grand Ave,199,Wabash Ave & Grand Ave,2019
3,25223643,2019-10-01 00:04:32,2019-10-01 00:43:43,313,Lakeview Ave & Fullerton Pkwy,290,Kedzie Ave & Palmer Ct,2019
4,25223644,2019-10-01 00:04:34,2019-10-01 00:35:42,210,Ashland Ave & Division St,382,Western Ave & Congress Pkwy,2019
...,...,...,...,...,...,...,...,...
1130936,ADDAA33CEBCAE733,2020-03-10 10:40:06,2020-03-10 10:40:07,675,HQ QR,675,HQ QR,2020
1130937,82B10FA3994BC66A,2020-03-07 15:25:55,2020-03-07 16:14:03,161,Rush St & Superior St,240,Sheridan Rd & Irving Park Rd,2020
1130938,AA0D5AAA0B59C8AA,2020-03-01 13:12:38,2020-03-01 13:38:29,141,Clark St & Lincoln Ave,210,Ashland Ave & Division St,2020
1130939,3296360A7BC20FB8,2020-03-07 18:02:45,2020-03-07 18:13:18,672,Franklin St & Illinois St,264,Stetson Ave & South Water St,2020


In [90]:


combined_df['started_at']= pd.to_datetime(combined_df['started_at'])
combined_df['ended_at'] = pd.to_datetime(combined_df['ended_at'])

In [92]:
combined_df.fillna({'end_station_id': 0}, inplace=True) 

In [93]:
schema = types.StructType([
    types.StructField("ride_id", types.StringType(), True),
    types.StructField("started_at", types.TimestampType(), True),
    types.StructField("ended_at", types.TimestampType(), True),
    types.StructField("start_station_id", types.LongType(), True),
    types.StructField("start_station_name", types.StringType(), True),
    types.StructField("end_station_id", types.LongType(), True),
    types.StructField("end_station_name", types.StringType(), True),
    types.StructField("year", types.LongType(), True),
     
])

In [86]:
spark = SparkSession.builder.appName("Exercise6").enableHiveSupport().getOrCreate()

In [94]:
sparkDF = spark.createDataFrame(combined_df, schema=schema)

In [95]:
sparkDF.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_id: long (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- end_station_id: long (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- year: long (nullable = true)



In [96]:
sparkDF.show()

+--------+-------------------+-------------------+----------------+--------------------+--------------+--------------------+----+
| ride_id|         started_at|           ended_at|start_station_id|  start_station_name|end_station_id|    end_station_name|year|
+--------+-------------------+-------------------+----------------+--------------------+--------------+--------------------+----+
|25223640|2019-10-01 00:01:39|2019-10-01 00:17:20|              20|Sheffield Ave & K...|           309|Leavitt St & Armi...|2019|
|25223641|2019-10-01 00:02:16|2019-10-01 00:06:34|              19|Throop (Loomis) S...|           241| Morgan St & Polk St|2019|
|25223642|2019-10-01 00:04:32|2019-10-01 00:18:43|              84|Milwaukee Ave & G...|           199|Wabash Ave & Gran...|2019|
|25223643|2019-10-01 00:04:32|2019-10-01 00:43:43|             313|Lakeview Ave & Fu...|           290|Kedzie Ave & Palm...|2019|
|25223644|2019-10-01 00:04:34|2019-10-01 00:35:42|             210|Ashland Ave & Div...|  